In [1]:
import sys
!{sys.executable} -m pip install cloudscraper
!{sys.executable} -m pip install selenium

In [2]:
#get url response
import requests
from bs4 import BeautifulSoup
import urllib
import cloudscraper
import json
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

#saves reviews in dataframe
import pandas as pd

In [3]:
#a means of connecting to a chrome server used to fully open the metacritic pages.
cService = webdriver.ChromeService(executable_path=r'C:/Users/DBeye/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe')

#used to access the metaritic page.

url = 'https://www.metacritic.com/game/yoshis-island-ds/critic-reviews/?platform=ds'

try:
  driver = webdriver.Chrome(service=cService)
  driver.get(url)

  #code to maximize window for long enough to scrape all
  # of the information.
  driver.maximize_window()
  
  
  #code to scroll down the page; scroll_num will be adjusted based on the number of reviews.
  scrollDown = 0
  #determines how far to scroll down for each page.
  scroll_num = 800
  #scrolls down the page until it reaches the end.
  
  time.sleep(10)
  while True:
    scrollDown += 1
    driver.execute_script("window.scrollBy(0,20)")
    if scrollDown > scroll_num:
      break
  """"""
  time.sleep(10)

finally:
  html = driver.page_source
  driver.quit()

In [4]:
#used to store review info.
review_dict = {'publication':[], 'rating':[], 'date_published':[], 'quote':[], 'platform':[], 'url_link':[]}

#used to parse the html code, find all of the review
soup = BeautifulSoup(html, 'html.parser')

#print(soup.find_all('div',class_="c-siteReview g-bg-gray10 u-grid g-outer-spacing-bottom-large"))

In [ ]:
#find all div elements containing the individual reviews.
for index, review in enumerate(soup.find_all('div',class_="c-siteReview g-bg-gray10 u-grid g-outer-spacing-bottom-large")):
  #print(review)
  review_info = {}
  #checks for reviews with "Read More" buttons.
  if review.find('div', class_='c-siteReview_quote g-outer-spacing-bottom-small'):
    #print(review.find('span', class_='c-globalButton_label').text)
    #print(review.find('div', class_='c-siteReview_quote g-outer-spacing-bottom-small').text)
    review_info['quote'] = review.find('div', class_='c-siteReview_quote g-outer-spacing-bottom-small').text.replace(",","~")
  #checks for reviews without "Read More" buttons.
  else:
    review_info['quote'] = "NO QUOTE"
  
  #Adds the review publication to the dictionary.
  if review.find('div',class_='c-siteReview_platform g-text-bold g-color-gray80 g-text-xsmall u-text-right u-text-uppercase'):
    review_info['publication'] = review.find('a',class_='c-siteReviewHeader_publicationName g-text-bold g-color-gray90').text.replace(",","~").replace("\n    ","").strip("  ")
  else:
    review_info['publication'] = "NO PUBLICATION"
  
  #Adds the numerical rating is available.
  if review.find('div',class_='c-siteReviewScore_background c-siteReviewScore_background-critic_medium'):
    review_info['rating'] = review.find('div',class_='c-siteReviewScore_background c-siteReviewScore_background-critic_medium').find('span').text
  else:
    review_info['rating'] = "tbd"
  
  #Adds the data published if available.
  if review.find('div',class_='c-siteReview_reviewDate g-color-gray80 u-text-uppercase'):
    #strips the empty spaces from the dates if they are found.
    review_info['date_published'] = review.find('div',class_='c-siteReview_reviewDate g-color-gray80 u-text-uppercase').text.replace("\n        ","").replace("\n      ","")
  else:
    #case where we cannot find any date.
    review_info['date_published'] = "DATE NOT FOUND"
  
  #Adds the platform if available.
  if review.find('div',class_='c-siteReview_platform g-text-bold g-color-gray80 g-text-xsmall u-text-right u-text-uppercase'):
    review_info['platform'] = review.find('div',class_='c-siteReview_platform g-text-bold g-color-gray80 g-text-xsmall u-text-right u-text-uppercase').text.replace(",","~").replace("\n      ","").replace("\n    ","")
  else:
    review_info['platform'] = "PLATFORM NOT FOUND"

  #adds the review link if available.
  if review.find('a',class_='c-siteReview_externalLink g-color-gray80 u-flexbox u-flexbox-alignCenter u-text-whitespace-nowrap'):
    review_info['url_link'] = review.find('a',class_='c-siteReview_externalLink g-color-gray80 u-flexbox u-flexbox-alignCenter u-text-whitespace-nowrap')['href']
  else:
    review_info['url_link'] = "NO LINK"
  
  review_dict[index+1] = review_info
    #['href']


"""
for review in soup.find_all('div', class_='c-siteReview_main g-inner-spacing-medium'):
  #checks for reviews with "Read More" buttons.
  if review.find('span', class_='c-globalButton_label'):
    review_dict['quote'].append(review.find('span', class_='c-globalButton_label').text)
    print(review.find('span', class_='c-globalButton_label').text)
  #checks for reviews without "Read More" buttons.
  else:
    review_dict['quote'].append(review.find('div',class_='c-siteReview_quote g-outer-spacing-bottom-medium').find('span').text)
    print(review.find('div',class_='c-siteReview_quote g-outer-spacing-bottom-medium').find('span').text)
"""

'\nfor review in soup.find_all(\'div\', class_=\'c-siteReview_main g-inner-spacing-medium\'):\n  #checks for reviews with "Read More" buttons.\n  if review.find(\'span\', class_=\'c-globalButton_label\'):\n    review_dict[\'quote\'].append(review.find(\'span\', class_=\'c-globalButton_label\').text)\n    print(review.find(\'span\', class_=\'c-globalButton_label\').text)\n  #checks for reviews without "Read More" buttons.\n  else:\n    review_dict[\'quote\'].append(review.find(\'div\',class_=\'c-siteReview_quote g-outer-spacing-bottom-medium\').find(\'span\').text)\n    print(review.find(\'div\',class_=\'c-siteReview_quote g-outer-spacing-bottom-medium\').find(\'span\').text)\n'

In [6]:
print(json.dumps(review_dict, indent=4))

{
    "1": {
        "quote": "It\u2019s improved in every way but still retains all of the elements that made the original an instant classic.",
        "publication": "Cheat Code Central",
        "rating": "96",
        "date_published": "\n",
        "platform": "DS",
        "url_link": "http://cheatcc.com/ds/rev/yoshisisland2review.html"
    },
    "2": {
        "quote": "Much better than most other games on the DS. In particular it puts New Super Mario Bros to shame.",
        "publication": "Jolt Online Gaming UK",
        "rating": "95",
        "date_published": "\n",
        "platform": "DS",
        "url_link": "http://www.jolt.co.uk/index.php?articleid=7670"
    },
    "3": {
        "quote": "Yoshi's Island DS is a terrific platformer that's every bit as fun and charming as its predecessor.",
        "publication": "GameSpot",
        "rating": "91",
        "date_published": "\n",
        "platform": "DS",
        "url_link": "http://www.gamespot.com/ds/action/yoshisisl

In [ ]:
import csv 

csv_filename = "null.csv"

fieldnames = ['publication', 'rating', 'date_published', 'quote', 'platform', 'url_link']

with open(csv_filename,mode='w',newline='') as file:
  writer = csv.DictWriter(file,fieldnames=fieldnames)
  writer.writeheader()
  for r in review_dict:
    writer.writerow({field: review_dict[r].get(field) or r for field in fieldnames})